In [31]:
import pandas as pd
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM,Dense,Embedding,Input,Dropout
from  keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras import optimizers
import keras
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import os
import time
from sklearn.utils import shuffle

In [5]:
dirs = os.listdir('./aclImdb/train')

In [6]:
text = []
labels = []
for i in dirs:
    if i == 'neg':
        for j in os.listdir('./aclImdb/train/neg'):
            with open('./aclImdb/train/neg/'+j, 'rb') as f:
                text.append(f.read())
                labels.append([1,0])
    if i == 'pos':
        for j in os.listdir('./aclImdb/train/pos'):
            with open('./aclImdb/train/pos/'+j, 'rb') as f:
                text.append(f.read())
                labels.append([0,1])

In [12]:
my_set = set()
for x in text:
    for i in str(x).split(' '):
        my_set.add(i)

In [10]:
len(my_set)

295309

In [13]:
word_dict = {}
for index, word in enumerate(my_set):
    word_dict[word] = index

In [14]:
def encode(text):
    vector = []
    for word in text.split(' '):
        vector.append(word_dict[word])
    return vector

In [11]:
encode(trainx[0])

[16789,
 8924,
 18123,
 6132,
 8399,
 13083,
 4958,
 13045,
 5820,
 15076,
 2388,
 7913,
 13083,
 8733,
 15076,
 13975,
 2388,
 7913,
 13083,
 2352,
 1077,
 17390,
 18123,
 15442,
 1321,
 11599,
 5188,
 5995,
 18123,
 15442,
 4285,
 5315,
 11955,
 18123,
 16893,
 9474,
 4005]

In [19]:
#vector takes fixed lenght i/p so we pad remaining
reviews = []
for review in text[:1000]:
    reviews.append(encode(str(review)))

In [20]:
reviews[:10]

[[124144,
  214730,
  175265,
  163115,
  189861,
  226887,
  241740,
  275505,
  268857,
  175265,
  11519,
  78509,
  211337,
  253999,
  175265,
  6945,
  248682,
  193020,
  14222,
  175265,
  214403,
  85455,
  214730,
  134229,
  166161,
  84694,
  9996,
  203976,
  127389,
  14222,
  155540,
  19428,
  10477,
  176784,
  243490,
  276712,
  266573,
  98615,
  125354,
  237500,
  214730,
  147509,
  235386,
  278207,
  201480,
  261513,
  134229,
  98615,
  45032,
  261472,
  253999,
  34286,
  275941,
  174893,
  214404,
  188371,
  201480,
  213153,
  234045,
  236142,
  91538,
  48331,
  81745,
  79548,
  98615,
  150795,
  28205,
  264294,
  155540,
  89817,
  59563,
  262513,
  23678,
  32226,
  116338,
  173900,
  38499,
  226940,
  232761,
  175265,
  107364,
  200067,
  228912,
  175265,
  110855,
  182965,
  147509,
  13685,
  62049,
  65243,
  149747,
  212739,
  253999,
  28320,
  49057,
  256389,
  266573,
  27186,
  45089,
  45118,
  245414,
  109676,
  290316,
  230

In [21]:
reviews = pad_sequences(reviews,maxlen=100,value=0.)

In [22]:
reviews[0]

array([211337, 253999, 175265,   6945, 248682, 193020,  14222, 175265,
       214403,  85455, 214730, 134229, 166161,  84694,   9996, 203976,
       127389,  14222, 155540,  19428,  10477, 176784, 243490, 276712,
       266573,  98615, 125354, 237500, 214730, 147509, 235386, 278207,
       201480, 261513, 134229,  98615,  45032, 261472, 253999,  34286,
       275941, 174893, 214404, 188371, 201480, 213153, 234045, 236142,
        91538,  48331,  81745,  79548,  98615, 150795,  28205, 264294,
       155540,  89817,  59563, 262513,  23678,  32226, 116338, 173900,
        38499, 226940, 232761, 175265, 107364, 200067, 228912, 175265,
       110855, 182965, 147509,  13685,  62049,  65243, 149747, 212739,
       253999,  28320,  49057, 256389, 266573,  27186,  45089,  45118,
       245414, 109676, 290316, 230342, 264997, 139141, 235692, 254348,
       209499, 264294, 197467, 188306])

In [33]:
reviews, labels = shuffle(reviews, labels[:1000])

# LSTM

In [35]:
adam = optimizers.Adam(lr=0.01)
model = Sequential()
model.add(Embedding(len(my_set)+1,64,input_length=100))
model.add(LSTM(64))
model.add(Dense(2,activation='softmax'))

model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
model.fit(reviews, np.array(labels[:1000]),batch_size=100,verbose=1,epochs=7)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 64)           18899840  
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 130       
Total params: 18,932,994
Trainable params: 18,932,994
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/7
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1755 - acc: 0.9460
Epoch 2/7
1000/1000 [==============================] - 6s 6ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 3/7
1000/1000 [==============================] - 6s 6ms/step - loss: 1.1921e-07 - acc: 1.0000
Epoch 4/7
1000/1000 [==============================] - 6s 6ms/step - loss: 1.1921e-07 - acc: 

In [36]:
model.save('lstm_1.h5')